## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [2]:
import datetime

import pandas as pd
from dotenv import load_dotenv

load_dotenv()
from web3 import Web3

from notebooks import get_block_by_ts
from notebooks import get_twap_bpt_price
from config_file import CORE_POOLS
from config_file import Chains
from config_file import WEB3_INSTANCES

TARGET_BLOCKS = {}
timestamp_now = 1695952610
timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
datetime_2_weeks_ago = datetime.datetime.fromtimestamp(timestamp_2_weeks_ago)

target_blocks = {}
bpt_twap_prices = {chain.value: {} for chain in Chains}
# Collect BPT prices for 2 weeks ago and now for all relevant pools and chains
for chain in Chains:
    target_blocks[chain.value] = (
        get_block_by_ts(timestamp_now, chain.value),  # Block now
        get_block_by_ts(timestamp_2_weeks_ago, chain.value)  # Block 2 weeks ago
    )
    print(f"Collecting bpt prices for {chain.value}")
    pools = CORE_POOLS.get(chain.value, None)
    if pools is None:
        continue
    for core_pool in pools:
        bpt_twap_prices[chain.value][core_pool] = get_twap_bpt_price(
            core_pool, chain.value, getattr(WEB3_INSTANCES, chain.value),
            start_date=datetime.datetime.fromtimestamp(timestamp_now), block_number=target_blocks[chain.value][0]
        )

KeyError: 'GNOSISNODEURL'

## Fetching fees and other data from the Balancer subgraphs

In [ ]:
from notebooks.fees_and_bribs.config_file import BALANCER_GRAPH_URLS
from notebooks import get_balancer_pool_snapshots
from typing import Dict

pool_snapshots = {}
for chain in Chains:
    pool_snapshots[chain.value] = (
        get_balancer_pool_snapshots(target_blocks[chain.value][0], BALANCER_GRAPH_URLS[chain.value]),  # now
        get_balancer_pool_snapshots(target_blocks[chain.value][1], BALANCER_GRAPH_URLS[chain.value]),  # 2 weeks ago
    )

## Extract fee data for CORE pools:


In [ ]:
from collections import defaultdict
from decimal import Decimal
from notebooks import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: Chains, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.datetime) -> tuple[dict, dict]:
    # Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
    # and then summing across all pools
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # Take first element of list, which is the most recent snapshot
        if not current_fees_snapshots or not fees_2_weeks_ago:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        bpt_price_usd = bpt_twap_prices[chain.value][pool]
        if bpt_price_usd is None:
            bpt_price_usd = 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT'] is not None and pool_snapshot_2_weeks_ago['pool'][
            'totalProtocolFeePaidInBPT'] is not None:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'])
            token_fees[pool_snapshot_now['pool']['id']].append({
                'symbol': pool_snapshot_now['pool']['symbol'],
                'token': pool_snapshot_now['pool']['symbol'],
                'token_fee': bpt_token_fee,
                'token_price': bpt_price_usd,
                'token_fee_in_usd': Decimal(bpt_token_fee) * bpt_price_usd,
                'token_addr': pool_snapshot_now['pool']['address'],
                'time_from': datetime_2_weeks_ago,
                'time_to': datetime_now,
                'chain': chain.value,
            })
        # Now collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
        else:
            bpt_price_usd = 0
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None))
                # Get twap token price from CoinGecko
                token_price = fetch_token_price_balgql(token_data['address'], chain.value, start_date) or 0
                token_fees[pool_snapshot_now['pool']['id']].append({
                    'symbol': pool_snapshot_now['pool']['symbol'],
                    'token': token_data['symbol'],
                    'token_fee': token_fee,
                    'token_price': token_price,
                    'token_fee_in_usd': Decimal(token_fee) * token_price if token_price is not None else 0,
                    'token_addr': token_data['address'],
                    'time_from': datetime_2_weeks_ago,
                    'time_to': datetime_now,
                    'chain': chain.value,
                })
        # Calculate non-BPT fees in USD
        fees[pool_snapshot_now['pool']['id']] = {
            'symbol': pool_snapshot_now['pool']['symbol'],
            'pool_addr': pool_snapshot_now['pool']['address'],
            'bpt_token_fee': round(bpt_token_fee, 2),
            # Get fee in USD by multiplying bpt_token_fee by price of BPT token taken from twap_bpt_price
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(sum([x['token_fee_in_usd'] for x in
                                            token_fees[
                                                pool_snapshot_now['pool']['symbol']]]) if bpt_price_usd == 0 else 0, 2),
            'time_from': datetime_2_weeks_ago,
            'time_to': datetime_now,
            'chain': chain.value,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']]
        }
    return fees


collected_fees = {}

for chain in Chains:
    core_pools = CORE_POOLS.get(chain.value, None)
    if not core_pools:
        continue
    collected_fees[chain.value] = collect_fee_info(core_pools, chain, pool_snapshots[chain.value][0],
                                                   pool_snapshots[chain.value][1], datetime_now)
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**collected_fees[Chains.MAINNET.value], **collected_fees[Chains.ARBITRUM.value],
              **collected_fees[Chains.POLYGON.value], **collected_fees[Chains.BASE.value],
              **collected_fees[Chains.AVALANCHE.value]}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

In [ ]:
# Remove `token_fees` field from dataframe, as it's too big
joint_fees_df_copy = joint_fees_df.drop(columns=['token_fees'])
# Display all rows in dataframe
pd.set_option('display.max_rows', 1000)
joint_fees_df_copy.sort_values(by=['chain'], ascending=False)

## Now let's calculate bribes paid to the pools and save them to csv

In [ ]:
from notebooks.fees_and_bribs.config_file import MIN_AURA_BRIB
from notebooks.fees_and_bribs.config_file import FEE_TAKEN
from notebooks.fees_and_bribs.config_file import REROUTE_CONFIG
from notebooks import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(WEB3_INSTANCES.mainnet, target_blocks[Chains.MAINNET.value][0])

# Bribes are split per chain and per pool, with each pool getting a share of the bribe proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the bribes. That 10% will be distributed between aura and vebal, proportional to their share of the bribe.


# TODO  Check that there are this many USD in the safe
FEES_TO_DISTRIBUTE = {
    Chains.MAINNET.value: Decimal(112945.77),
    Chains.ARBITRUM.value: Decimal(39481.82),
    Chains.POLYGON.value: Decimal(6592.87),
    Chains.BASE.value: Decimal(10260.35),
    Chains.GNOSIS.value: Decimal(2503.21),
    Chains.AVALANCHE.value: Decimal(10297),
}


# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_bribes(fees: Dict, chain: str, fees_to_distribute: Decimal) -> Dict[str, Dict]:
    pool_bribs = {}
    dao_fess += 0.15 * FEES_TO_DISTRIBUTE
    vebal += 0.35 * FEES_TO_DISTRIBUTE
    bribe = 0.5 * FEES_TO_DISTRIBUTE
    
    # Calculate pool share in fees
    bpt_fees = sum([data['bpt_token_fee_in_usd'] for pool, data in fees.items()])
    token_fees = sum([data['token_fees_in_usd'] for pool, data in fees.items()])
    total_fees = bpt_fees + token_fees
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        pool_share = pool_fees / Decimal(total_fees)
        # If aura bribes is less than 500 USDC, we pay all bribes to balancer
        aura_bribes = round(pool_share * fees_to_distribute * aura_vebal_share * FEE_TAKEN, 2)
        if aura_bribes <= Decimal(MIN_AURA_BRIB):
            aura_bribes = Decimal(0)
            bal_bribes = round(pool_share * fees_to_distribute * FEE_TAKEN, 2)
        else:
            bal_bribes = round(pool_share * fees_to_distribute * (1 - aura_vebal_share) * FEE_TAKEN, 2)

        # Split fees between aura and bal fees
        pool_bribs[pool] = {
            "symbol": data['symbol'],
            "chain": chain,
            "aura_bribes": aura_bribes,
            "bal_bribes": bal_bribes,
            "protocol_fees_collected": pool_fees,
            "pool_addr": data['pool_addr'],
        }
    return pool_bribs

def re_route_bribs(bribes_data: Dict[str, Dict], chain: Chains) -> Dict[str, Dict]:
    """
    If pool is in re-route configuration, all bribes from that pool should be distributed to destination pool
      Ex: {source_pool: destination_pool}
    """
    if not chain.value in REROUTE_CONFIG:
        return bribes_data
    for pool_id, _data in bribes_data.items():
        if pool_id in REROUTE_CONFIG[chain.value]:
            # Re route everything to destination pool and set source pool bribes to 0
            bribes_data[REROUTE_CONFIG[pool_id]]['aura_bribes'] += _data['aura_bribes']
            bribes_data[REROUTE_CONFIG[pool_id]]['bal_bribes'] += _data['bal_bribes']
            bribes_data[pool_id]['aura_bribes'] = 0
            bribes_data[pool_id]['bal_bribes'] = 0
    return bribes_data


bribes = {}
for chain in Chains:
    if not collected_fees.get(chain.value):
        continue
    _bribs = calc_and_split_bribes(collected_fees[chain.value], chain.value,
                                                FEES_TO_DISTRIBUTE[chain.value])
    # Reroute bribes if needed
    bribs = re_route_bribs(_bribs, chain)
    bribes[chain.value] = bribs
    
joint_bribes_data = {**bribes[Chains.MAINNET.value], **bribes[Chains.ARBITRUM.value],
                     **bribes[Chains.POLYGON.value], **bribes[Chains.BASE.value], **bribes[Chains.AVALANCHE.value]}
# Sort by chain:
joint_bribes_data = {k: v for k, v in sorted(joint_bribes_data.items(), key=lambda item: item[1]['chain'])}
joint_bribes_df = pd.DataFrame.from_dict(joint_bribes_data, orient='index')
# Sort by chain
# Dump into csv and prefix with dates
joint_bribes_df.to_csv(f'./output/bribes_{datetime_2_weeks_ago.date()}_{datetime_now.date()}.csv')
joint_bribes_df

## Compare data from the sheet with the data we collected
What does table below represent:
- aura_automated_bribe: bribes paid to aura per pool calculated in this notebook
- aura_actual_bribe: actual bribes paid to aura per pool from the sheet
- aura_bribe_delta: difference between aura_automated_bribe and aura_actual_bribe
- aura_bribe_delta_%: difference between aura_automated_bribe and aura_actual_bribe in %

Same thing for balancer side:
- bal_automated_bribe: bribes paid to balancer per pool calculated in this notebook
- bal_actual_bribe: actual bribes paid to balancer per pool from the sheet
- bal_bribe_delta: difference between bal_automated_bribe and bal_actual_bribe
- bal_bribe_delta_%: difference between bal_automated_bribe and bal_actual_bribe in %

In [ ]:
from notebooks import fetch_all_pools_info

mapped_gauges = {}

for chain in Chains:
    pools_info = fetch_all_pools_info(chain.value)
    mapped_gauges[chain.value] = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for
                                  pool_info in pools_info}

In [ ]:
# Load csv sheet to compare with the data collected above:
bribes_df = pd.read_csv('compare_sheet.csv')
# To dict:
bribes_to_compare = bribes_df.to_dict(orient='records')
# Each dict in list needs target to be Web3.toChecksumAddress
bribes_csv_checksummed = []
for bribe in bribes_to_compare:
    bribes_csv_checksummed.append({
        'target': Web3.to_checksum_address(bribe['target']),
        'platform': bribe['platform'],
        'amount': bribe['amount'],
    })
# These are bribes paid de-facto, so we need to convert them to the same format as we have in our data
bribes_final_csv = {}
for bribe in bribes_csv_checksummed:
    if bribe['target'] not in bribes_final_csv:
        bribes_final_csv[bribe['target']] = {}
    bribes_final_csv[bribe['target']][bribe['platform']] = Decimal(bribe['amount'])

In [ ]:
# Now we need to compare the data we collected with the data from the sheet
# We need to compare bribes paid to each pool
joint_bribes_dict = joint_bribes_df.to_dict(orient='records')
bribes_delta = {}
for gauge, data in bribes_final_csv.items():
    for automated_bribe in joint_bribes_dict:
        filtered_gauges = mapped_gauges[automated_bribe['chain']]
        # If this condition is true, we can compare the bribes
        if filtered_gauges[automated_bribe['pool_addr']] == gauge:
            # Calculate delta between automated bribes and bribes from the sheet
            # Calculate delta between automated_bribe['aura_bribes'] and data['aura'] in %
            if automated_bribe['aura_bribes'] > data['aura']:
                aura_bribe_delta = automated_bribe['aura_bribes'] - data['aura']
                aura_bribe_delta_pct = (automated_bribe['aura_bribes'] - data['aura']) / data['aura'] * 100 if \
                    data['aura'] != 0 else 0
            else:
                aura_bribe_delta = data['aura'] - automated_bribe['aura_bribes']
                aura_bribe_delta_pct = (data['aura'] - automated_bribe['aura_bribes']) / data['aura'] * 100 if \
                    data['aura'] != 0 else 0
            if automated_bribe['bal_bribes'] > data['balancer']:
                bal_bribe_delta = automated_bribe['bal_bribes'] - data['balancer']
                bal_bribe_delta_pct = (automated_bribe['bal_bribes'] - data['balancer']) / data['balancer'] * 100 if \
                    data['balancer'] != 0 else 0
            else:
                bal_bribe_delta = data['balancer'] - automated_bribe['bal_bribes']
                bal_bribe_delta_pct = (data['balancer'] - automated_bribe['bal_bribes']) / data['balancer'] * 100 if \
                    data['balancer'] != 0 else 0
            bribes_delta[gauge] = {
                'symbol': automated_bribe['symbol'],
                'chain': automated_bribe['chain'],
                'aura_automated_bribe': automated_bribe['aura_bribes'],
                'aura_actual_bribe': round(data['aura'], 2),
                'aura_bribe_delta': round(aura_bribe_delta, 2),
                'aura_bribe_delta_%': f'{round(aura_bribe_delta_pct, 2)}%',
                'bal_automated_bribe': automated_bribe['bal_bribes'],
                'bal_actual_bribe': round(data['balancer'], 2),
                'bal_bribe_delta': round(bal_bribe_delta, 2),
                'bal_bribe_delta_%': f'{round(bal_bribe_delta_pct, 2)}%',
            }
# To df and print out without index:
bribes_delta_df = pd.DataFrame.from_dict(bribes_delta, orient='index')
bribes_delta_df.sort_values(by=['chain', 'aura_bribe_delta'], ascending=False)
# Also dump to csv
bribes_delta_df.to_csv(f'./output/bribes_delta_{datetime_2_weeks_ago.date()}_{datetime_now.date()}.csv')
bribes_delta_df